In [6]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import json
# 鍵とnonceを生成
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12) # GCM推奨サイズ
out={"original":[],"encrypted":[]}


    

In [ ]:
def encrypt(plaintext:str|bytes,key,is_str=True):
    nonce = get_random_bytes(12)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    aad = b"header-data"  # 認証のみ対象にする追加データ
    cipher.update(aad)
    #original="secret message"*i
    if is_str:
        plaintext = (plaintext).encode('utf-8')
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)

    # 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
    encrypted_package = nonce + ciphertext + tag
    print("暗号化パッケージ:", encrypted_package)

    # 復号化
    nonce_dec = encrypted_package[:12]
    ciphertext_dec = encrypted_package[12:-16]
    tag_dec = encrypted_package[-16:]

    cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
    cipher_dec.update(aad)
    #decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)
    #print("復号結果:", decrypted)
    import base64
    encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
    return encoded_str
# 暗号化

In [2]:
for i in [60,70,80,90]:
    original="secret message"*i
    cipher=encrypt(original,key)

    print(len(original))
    out["original"].append(len(original))
    out["encrypted"].append(len(cipher))


暗号化パッケージ: b'\x81\xb6\x14\xc8 \x7f\xe3U\x98\xd5s~r\xf2\xc6*\x05O\xd5\x99k_)[\xda\xdd\xdf\xd2\xf6\x1d\xc9\xa5\xa1\xee,\xac\xcc\xaaA(\xd8\x1d\xe0G\xbc6\x15Gj\xc1\xb3\xd5!\xec\xab=f\xed)\x9f\xfe\xc7\xef\x08\xd1\xef\x81\x8a~\x075\x18X\x8d\xcf\xf1nj\xe9\xc5D\x9a+\xd2\xab\xb3E\x0e\x81\xf6\xf2r}\xf30\'\x1e\x91\xc2\xd4<Xx\x15\xdcD\xee\x996\xbf\x10\x9f\x9a\xa7\xb7\xce3$\x19U\x89"e\xf1w\xd1\x81\xf1\x9f\xe1\x95\xdbd\xba\xdfn\xb8\xa8c(\x13\xcfT\xb9\x94_B\xd7\\\xba\xc5RJ|6S\x81p9\x94\xa1w\x9diF\xc3x\x81>2\xe1"\xe4P66\xdb.\xec\x86\x83\x94\x13v\x87n\xb1_\xf1\xd7\xf8\xa7=\xcf\xcc"\xab\xcc|\xa6^\x1eY\xbdNLX\xbd\x05\xbf\xe7\x1bS\xe9\xc5\x06\xa3\xf5\x97HJ\xd2\x0b\xc2\xf4\xb73AJ\xc5\xe8\xbc\xca\x18}\xcc\x9c\x14)qd\xa0\xf1M1\x16i\x8bi\xc3\xb1\xeaw#?\xdf\xdaL\x11:P\xe5W`\xa4\xd9-\xde\xd0i\xbeY\xec2\x90\xa7\x15\x13\xc3\xcf\xd9\x10\xfd\xceiO\x8a\xf8\xf6\xcc\x11\xf2jIZ\xa2\x08nP\nT\x7f\x8a\x9d\xd3y\x90\xa6\xa8\x0c\x90WX\xdc\xc3\xb7[\x01\r\x03eg\xf4%\xf1k\xca\xeaI\xc2\xe9\x90\xa3\x1c\xd0\x98\xe4\xf6\x9dW=\xce\xc

In [3]:
import pandas as pd

pd.DataFrame(out)


,original,encrypted
0,840,1160
1,980,1344
2,1120,1532
3,1260,1720


In [ ]:
import gzip
import base64
import urllib.parse
import json

# 圧縮したい文字列
tmp={
  "status": "success",
  "timestamp": "2025-11-25T09:09:00Z",
  "user": {
    "id": 12345,
    "name": "Dummy User",
    "email": "dummy@example.com",
    "roles": ["admin", "editor", "viewer"],
    "preferences": {
      "language": "ja",
      "timezone": "Asia/Tokyo",
      "notifications": {
        "email": True,
        "sms": False,
        "push": True
      }
    }
  },
  "data": {
    "items": [
      {
        "id": "A001",
        "title": "sample 1",
        "description": "This is dummy data created for testing JSON responses.",
        "tags": ["test", "dummy", "json"],
        "metrics": {
          "views": 120,
          "likes": 45,
          "comments": 10
        }
      }
    ],
    "summary": {
      "total_items": 2,
      "total_views": 350,
      "total_likes": 112,
      "total_comments": 35
    }
  },
 
}
#text = "これはテスト用の文字列です"
text=json.dumps(tmp, ensure_ascii=False)
# 1. UTF-8バイト列に変換
data = text.encode("utf-8")

# 2. gzip圧縮
compressed = gzip.compress(data,compresslevel=9)

# 3. Base64 (URL-safe) エンコード
encoded = base64.urlsafe_b64encode(compressed).decode("utf-8")

# 4. クエリパラメータに組み込む
params = {"q": encoded}
#print(len(base64.urlsafe_b64encode(text.encode("utf-8"))))
print(len(text))
print(len(encoded))
query_string = urllib.parse.urlencode(params)
url = f"https://example.com/search?{query_string}"
print(url)

816
611
488
https://example.com/search?q=H4sIAJgGJWkC_01RwU7DMAz9lShnNtqOHtiJSZw4wIFxYUIoat0uW5NUcQqMqf-O7a6AFKnpe8_P9stZYzJpQL1WGoeqAkR9pXSyDohwPeNFVpSLPF8U5Ta7XfPJXlk0IETiz9rW9MmL1U1JqDcOuOp-cO6kXlhDKDhjO4Zrhu_gi7w7WFbBMRtDBzzCTpvaWS8FtU1BSj8sfJLJG137CA1E8JWoz7ozvh1MK_0OZh78O3hBNmjN9TYcT4EZH5JtbGWSDX6qnmdKcQASoGO4MR3yXz_g_sKN40hAbZKZtk0gyt1lcb3JsnzqnTppjLKcErAGrKLtuStT271FRUdiUOypqggmQa2aEFWi1K1v1cPz06OKgD3NCrgUd9NOEbFGnNmCLwckb47HQYq2mpbj1FCeJSOms0eJTF6IQnfgk7DZOHIlkpeJJ6lMIZnufd6y4NaCzI6rMvvFZt88_9P9s1-VlN0PVQlANWMCAAA%3D


In [6]:
len(text)

884

In [25]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12)
cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
aad = b"header-data"  # 認証のみ対象にする追加データ
cipher.update(aad)
#original="secret message"*i

plaintext =data# (original).encode('utf-8')
ciphertext, tag = cipher.encrypt_and_digest(plaintext)

# 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
encrypted_package = nonce + ciphertext + tag
print("暗号化パッケージ:", encrypted_package)

# 復号化
nonce_dec = encrypted_package[:12]
ciphertext_dec = encrypted_package[12:-16]
tag_dec = encrypted_package[-16:]

cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
cipher_dec.update(aad)
decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)

print("復号結果:", decrypted)
import base64
encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
print("Base64文字列:", encoded_str)
print(len(encoded_str))
#print(len(original))
#out["original"].append(len(original))
#out["encrypted"].append(len(encoded_str))

暗号化パッケージ: b';8\x89\x7f\xac\xcf\xe8l\xbb\xcf\x98\xae\xa7\x07y&%yT\x00\xf3\xdeS\xb6\x0bT\xbf\x00\x17eQ\x9b\x87\xf9\xa2z\xdd\xc7\x8f)\x157\x858W\\\xb6\xae\t\x89\x15\x17\xd1~\xdaa\x1fhT\x99\xa6\xf2\xac\xd3 \xa1\xa1\xd9\xaa\xbe\x8e\x89\xd1v\x1b\xbbHW\xaaK#\x92Du&\xd1\x87\xfe\x9cm\xca\x07\xe8o\xad\x1cZ\xa7)\x88\xb6p\x06\xaaW\xe7\xd4R\xe5b\xce\x86\xe8\x1ev,\xfe\xc9\x87\xab,\n\xf4P!\xd1\xb4\xa6\xcf\x98\xca\xa6\x88\x18J)\xa6\xa3cm\xc0\xb3\xf6$-\'n[\rL\x93\xff\xfc\x93\x92\xc3\xd4\x8d\x8b\x876\xf6\x91\x03W\xda\xe6\x7fa\xf2R\x7f}\xa0v3O"7\xc7\xe3\x81\xa1\xd2\x05\xd9d\x96\x89\xdbRn\xfeS)DEX\x18\xe6\x14\xf2\xd6\xa3fZ\xdc(\xe7\xde@[\xb6\xa8\xad\x12\x1dNE\xd6\x98\x18\x10\xb5\xed\\\xb8\xf4\x853\xcevw\x16zq\xc1\xd2\xb9+zyB\x8e\xd0\xe7\t\xd6}\x87\xc0\xdeS\x96\x9aEF}\xba\x8a\x85ib\xd1\xb9~x\xc7\x9f\xfd\xfb\xd9;\xcd%\xc4\xe1.\xeb\x89\x05\xcf\xea\t<\x18\xc4,\xe2\xc5\xa2\x7f\xb8`Zj\x99\xa6\x1b\xf8\r%o \xe4\x87\x05*$T\x98\xf5s\xae\x0c\xf0gh\'\x8b)\'m\x07\x8c\xa3\x96\x9e\x90\x01$\xceB\x14\xbe|\x13\x19\\!H\xd6"

In [9]:
data = {
"group_id": "m_ehime_pref",
"images": [
{"products": [{"pk": 144, "displayed": False}], "version_pk": None, "result": {"pk": 201, "displayed": True}},
{"products": [{"pk": 143, "displayed": False}], "version_pk": None, "result": {"pk": 200, "displayed": False}},
],
"monitoring_points": [{"pk": 302, "displayed": True}],
"thematics": [
{"pk": 37, "displayed": True},
{"pk": 39, "displayed": True},
{"pk": 42, "displayed": True},
{"pk": 38, "displayed": True},
{"pk": 40, "displayed": True},
{"pk": 41, "displayed": True},
],
"view": {
"monitoring_points": [{"pk": 302, "filter": "FIT_ID:a0BIm000000QbCJMA0", "activate_tab": True}],
"filtered_result_one": True,
},
}
text=json.dumps(data, ensure_ascii=False)
# 1. UTF-8バイト列に変換
data = text.encode("utf-8")

# 2. gzip圧縮
compressed = gzip.compress(data,compresslevel=9)

# 3. Base64 (URL-safe) エンコード
encoded = base64.urlsafe_b64encode(compressed).decode("utf-8")

In [12]:
print(len(encoded))
print(len(text))
print(type(compressed))

344
655
<class 'bytes'>
